In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

MNIST Dataset - Train a model with 99.8% accuracy on Handwritten Digits

##Dataset Details
MNIST dataset has items of handwriting -- the digits 0 through 9. It has grey scale images, each of size 28X28 pixels. Each pixel in a grey scale image is of size 1 byte, so there are a total of 784 bytes.

##Goal of the Model
The goal is to train an MNIST classifier that trains to 99% accuracy or above, and does it without a fixed number of epochs -- i.e. to stop training once you reach that level of accuracy.

When it reaches 99.8% or greater it should print the following string to the console and stop training the model. "Reached 99.8% accuracy so cancelling training!".

##Solution
1. Use callbacks to stop training once the desired accuracy is reached
2. Use CNNs to improve the Image classification model.


In [5]:
import tensorflow as tf
import matplotlib.pyplot as plt

#Define the callback - Stop training if accuracy reaches 99.8%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.998):
      print('Reached 99.8% accuracy, so cancelling training')
      self.model.stop_training = True

# Load data from MNIST Keras API
mnist = tf.keras.datasets.mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()

# Instantiate the callback
callbacks = myCallback()

#Reshape the input to a 4D Tensor as the Convolution expects the input as (batch_size, height, width, depth)
training_images = training_images.reshape(60000, 28, 28, 1)
test_images = test_images.reshape(10000, 28, 28, 1)

#Normalize the input data
training_images = training_images/255.0
test_images = test_images/255.0


# Define the hidden layers of the model
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
                                    tf.keras.layers.MaxPool2D((2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(units=256, activation='relu'),
                                    tf.keras.layers.Dense(units=128, activation='relu'),
                                    tf.keras.layers.Dense(units=10, activation='softmax')
])

# Define the optimizer and loss function to be used by the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics = 'accuracy')

#Print the model summary
model.summary()

# Train the model
model.fit(training_images, training_labels, epochs=20, callbacks=[callbacks])

#Test the model
model.evaluate(test_images, test_labels)


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 13, 13, 64)        0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 10816)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 256)               2769152   
_________________________________________________________________
dense_13 (Dense)             (None, 128)               32896     
_________________________________________________________________
dense_14 (Dense)             (None, 10)                1290      
Total params: 2,803,978
Trainable params: 2,803,978
Non-trainable params: 0
____________________________________________

[0.06828329712152481, 0.9873999953269958]